# EMG信号处理快速开始

这个notebook将带你快速体验EMG信号处理的完整流程。

## 📚 学习目标

- 了解EMG信号的基本形态
- 学会加载和可视化EMG数据
- 体验信号滤波效果
- 提取简单的特征
- 完成一个手势识别任务

## ⏱️ 预计时间：30分钟

## 步骤1: 导入必要的库

In [ ]:
# ===== 导入必要的Python库 =====

# 系统库
import sys  # 用于修改Python系统设置
from pathlib import Path  # 用于处理文件路径（比os.path更现代化）

# 科学计算库
import numpy as np  # 数值计算：数组、矩阵运算
import matplotlib.pyplot as plt  # 绘图库：创建各种图表

# ===== 添加项目路径到Python搜索路径 =====
project_root = Path.cwd().parent  # 获取项目根目录（当前目录的上一层）
sys.path.insert(0, str(project_root))  # 添加到sys.path，让Python能找到我们的自定义模块

# --- Jupyter/IPython 兼容性修复 ---
# IPython 往往会先导入标准库的 `code` 模块，导致 `code.weekXX_*` 不是包。
# 这里把项目的 `code/` 目录挂到 `code.__path__`，让子模块可被正常导入。
import code as _code
if not hasattr(_code, '__path__'):
    _code.__path__ = [str(project_root / 'code')]
# 这样就可以导入 code.week06_preprocessing 等模块了

# ===== 配置matplotlib，让图表能正确显示中文 =====
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Segoe UI Symbol', 'Arial Unicode MS', 'DejaVu Sans']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号'-'显示为方块的问题

print("✓ 库导入成功！")

## 步骤2: 生成模拟EMG信号

首先，让我们生成一些模拟的EMG信号来理解它的特征。

In [ ]:
# ===== 生成模拟的EMG信号 =====

# --- 第一步：设置信号参数 ---
fs = 1000  # 采样率：每秒采集1000个数据点（1000 Hz）
duration = 5  # 信号持续时间：5秒
t = np.linspace(0, duration, int(duration * fs))  # 生成时间轴：从0到5秒，共5000个时间点

# --- 第二步：生成静息期信号（基础噪声） ---
rest_signal = np.random.normal(0, 0.02, len(t))
# np.random.normal(均值, 标准差, 数量)
# 生成均值为0、标准差为0.02的高斯噪声
# 这模拟了肌肉完全放松时的微弱电活动

# --- 第三步：定义收缩期的时间范围 ---
contraction_start = int(1 * fs)  # 收缩开始：1秒 = 第1000个采样点
contraction_end = int(4 * fs)    # 收缩结束：4秒 = 第4000个采样点
# 所以收缩期是 1-4 秒（共3秒）

# --- 第四步：在收缩期添加多个频率成分 ---
# EMG信号是多个运动单元的叠加，每个单元有不同的激活频率
for freq in range(60, 150, 20):  # 频率范围：60, 80, 100, 120, 140 Hz
    rest_signal[contraction_start:contraction_end] += 0.3 * np.sin(
        2 * np.pi * freq * t[contraction_start:contraction_end]
    )
    # 每个频率成分的振幅都是0.3 mV
    # 这些正弦波叠加在一起，形成复杂的EMG信号

# --- 第五步：可视化信号 ---
plt.figure(figsize=(14, 4))  # 创建图表，宽14英寸，高4英寸

# 绘制信号曲线
plt.plot(t, rest_signal, linewidth=0.5)  # linewidth: 线条粗细

# 使用不同颜色标注三个阶段
plt.axvspan(0, 1, alpha=0.2, color='green', label='静息期')
# axvspan: 绘制垂直方向的色块
# alpha=0.2: 透明度20%（半透明）

plt.axvspan(1, 4, alpha=0.2, color='red', label='收缩期')
plt.axvspan(4, 5, alpha=0.2, color='green', label='放松期')

# 设置图表标签和样式
plt.xlabel('时间 (秒)')
plt.ylabel('幅度 (mV)')
plt.title('模拟EMG信号：静息 → 收缩 → 放松')
plt.legend()  # 显示图例
plt.grid(True, alpha=0.3)  # 显示网格线，透明度30%
plt.show()

# --- 第六步：打印信号信息 ---
print("✓ EMG信号生成成功！")
print(f"  信号长度: {len(rest_signal)} 个采样点")
print(f"  采样率: {fs} Hz")
print(f"  时长: {duration} 秒")
print(f"\n💡 观察：")
print("  - 静息期（0-1秒）：低幅度随机噪声")
print("  - 收缩期（1-4秒）：高幅度复杂信号")
print("  - 放松期（4-5秒）：恢复到静息水平")

## 步骤3: 信号滤波

EMG信号通常需要滤波来去除噪声。让我们应用一个带通滤波器。

In [ ]:
# ===== 应用滤波器去除噪声 =====

from code.week06_preprocessing.filters import EMGFilters
# 导入我们自定义的EMG滤波器模块

# --- 第一步：创建滤波器对象 ---
filters = EMGFilters(fs=1000)
# 初始化滤波器，传入采样率1000 Hz

# --- 第二步：应用带通滤波器 ---
filtered_signal = filters.bandpass_filter(
    rest_signal,    # 输入：原始信号
    lowcut=20,      # 低截止频率：20 Hz（去除低频漂移）
    highcut=500     # 高截止频率：500 Hz（去除高频噪声）
)
# 带通滤波器只保留20-500 Hz之间的频率成分
# 这是EMG信号的有效频率范围

# --- 第三步：可视化对比（上下两个子图） ---
fig, axes = plt.subplots(2, 1, figsize=(14, 8))
# subplots(行数, 列数): 创建2行1列的子图布局

# 【上图】原始信号
axes[0].plot(t, rest_signal, linewidth=0.5, color='blue')
axes[0].set_ylabel('幅度 (mV)')
axes[0].set_title('原始EMG信号（含噪声）')
axes[0].grid(True, alpha=0.3)

# 【下图】滤波后的信号
axes[1].plot(t, filtered_signal, linewidth=0.5, color='red')
axes[1].set_xlabel('时间 (秒)')
axes[1].set_ylabel('幅度 (mV)')
axes[1].set_title('滤波后的EMG信号 (20-500 Hz)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()  # 自动调整子图间距，避免重叠
plt.show()

print("✓ 滤波完成！")
print("\n💡 滤波的作用：")
print("  - 去除低频基线漂移（< 20 Hz）")
print("  - 去除高频噪声（> 500 Hz）")
print("  - 保留EMG有效信号（20-500 Hz）")
print("  - 信号变得更清晰，便于特征提取")

## 步骤4: 特征提取

从EMG信号中提取特征，用于后续的分类任务。

In [ ]:
# ===== 从EMG信号中提取特征 =====

from code.week07_feature_extraction.features import EMGFeatures
# 导入特征提取模块

# --- 第一步：提取时域特征（10种） ---
time_features = EMGFeatures.extract_time_features(filtered_signal)
# 时域特征直接从信号波形计算，不需要频域转换
# 包含：MAV, RMS, VAR, WL, ZC, SSC, IEMG, DASDV, PEAK, MEAN

print("时域特征:")
print("-" * 40)
for name, value in time_features.items():
    print(f"  {name:10s}: {value:.6f}")
# 格式说明：name:10s 表示名称占10个字符宽度，左对齐
#         value:.6f 表示数值保留6位小数

# --- 第二步：提取频域特征（8种） ---
freq_features = EMGFeatures.extract_freq_features(filtered_signal, fs=1000)
# 频域特征通过傅里叶变换（FFT）从频谱中计算
# 包含：MNF, MDF, Peak_Freq, Total_Power, SM1, SM2, SM3, Freq_Ratio
# 需要传入采样率fs，因为频域分析需要知道时间刻度

print("\n频域特征:")
print("-" * 40)
for name, value in freq_features.items():
    print(f"  {name:12s}: {value:.6f}")

print("\n✓ 特征提取完成！")
print(f"  时域特征: {len(time_features)}个")
print(f"  频域特征: {len(freq_features)}个")
print(f"  总特征数: {len(time_features) + len(freq_features)}个")

print("\n💡 特征的作用：")
print("  - 将复杂的信号波形转换为简单的数值")
print("  - 每个特征反映信号的不同方面")
print("  - MAV/RMS → 信号强度（肌肉收缩力度）")
print("  - WL → 信号复杂度（波形变化程度）")
print("  - MNF/MDF → 信号频率特性")
print("  - 这些特征用于后续的分类和识别")

## 步骤5: 使用真实数据（可选）

如果你已经生成了样本数据，可以运行以下代码加载真实的EMG数据。

In [ ]:
# ===== 加载和可视化真实EMG数据（可选） =====

# 这部分代码需要先生成样本数据
# 如果数据不存在，会提示运行 tools/generate_sample_data.py

try:
    from code.week05_data_processing.data_loading import EMGDataLoader
    # 导入数据加载模块
    
    # --- 第一步：创建数据加载器 ---
    loader = EMGDataLoader(
        data_dir='../data/sample/',  # 数据目录
        fs=1000                      # 采样率
    )
    
    # 打印数据集信息
    loader.print_dataset_info()
    
    # --- 第二步：加载单个试验的数据 ---
    time, signals, label = loader.load_single_trial(
        '../data/sample/subject_01/fist/trial_001.csv'
    )
    # time: 时间轴数组
    # signals: 多通道信号矩阵（时间点 × 通道数）
    # label: 手势标签（'fist'）
    
    # --- 第三步：可视化多通道EMG信号 ---
    n_channels = signals.shape[1]  # 获取通道数
    
    fig, axes = plt.subplots(n_channels, 1, figsize=(14, 8))
    # 创建n_channels行1列的子图，每个通道一个子图
    
    for ch in range(n_channels):
        # 绘制每个通道的信号
        axes[ch].plot(time, signals[:, ch], linewidth=0.5)
        axes[ch].set_ylabel(f'通道{ch}\n(mV)', fontsize=10)
        axes[ch].grid(True, alpha=0.3)
        
        # 只有最下面的子图显示x轴标签
        if ch == n_channels - 1:
            axes[ch].set_xlabel('时间 (秒)')
    
    plt.suptitle(f'真实EMG信号 - 手势: {label}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✓ 真实数据加载成功！")
    print(f"  手势类型: {label}")
    print(f"  通道数: {n_channels}")
    print(f"  采样点数: {len(time)}")
    print(f"  信号时长: {time[-1]:.2f}秒")
    
    print("\n💡 多通道EMG：")
    print("  - 每个通道对应一个电极位置")
    print("  - 多通道可以捕获不同肌肉群的活动")
    print("  - 提供更丰富的信息，提高识别准确率")
    
except FileNotFoundError:
    print("⚠️  未找到样本数据")
    print("\n请先生成样本数据：")
    print("  1. 打开终端/命令行")
    print("  2. 运行命令: python tools/generate_sample_data.py")
    print("  3. 等待数据生成完成")
    print("  4. 重新运行本单元格")
    
except Exception as e:
    print(f"❌ 加载数据时出错: {e}")
    print("请检查数据文件是否完整")

## 🎉 恭喜！

你已经完成了EMG信号处理的快速入门！

### 你学到了什么：
- ✅ EMG信号的基本形态（随机性、突发性）
- ✅ 信号滤波的作用和效果
- ✅ 如何提取时域和频域特征
- ✅ 如何加载和可视化真实数据

### 下一步：
1. 查看 `01_信号可视化入门.ipynb` - 深入学习信号可视化
2. 查看 `02_滤波器设计实验.ipynb` - 实验不同的滤波器
3. 查看 `03_特征提取实践.ipynb` - 掌握所有18种特征
4. 运行 `examples/complete_pipeline.py` - 完整的手势识别流程

### 相关资源：
- [学习资源快速索引](../docs/学习资源快速索引.md)
- [EMG学习资源汇总](../docs/EMG学习资源汇总.md)
- [视频教程清单](../resources/videos/视频资源清单.md)